In [2]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [4]:
# Load the movie and credits datasets
movies_df = pd.read_csv("tmdb_5000_movies.csv")
credits_df = pd.read_csv("tmdb_5000_credits.csv")

# Display first few rows of both datasets
print(movies_df.head())
print(credits_df.head())


      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [20]:
import pandas as pd

# Load datasets
movies_df = pd.read_csv("tmdb_5000_movies.csv")
credits_df = pd.read_csv("tmdb_5000_credits.csv")

# Print column names to check if 'id' exists
print("Movies Dataset Columns:", movies_df.columns)
print("Credits Dataset Columns:", credits_df.columns)

# Rename 'movie_id' to 'id' in credits_df
credits_df.rename(columns={'movie_id': 'id'}, inplace=True)

# Merge on 'id'
movies_df = movies_df.merge(credits_df, on='id', how='inner')

# Print columns to verify
print(movies_df.columns)

# Rename 'title_x' as 'title' if needed
if 'title_x' in movies_df.columns:
    movies_df.rename(columns={'title_x': 'title'}, inplace=True)

# Keep only required columns
movies_df = movies_df[['id', 'title', 'overview', 'genres', 'cast', 'crew']]

# Drop missing values
movies_df.dropna(inplace=True)

# Display dataset info
movies_df.info()


Movies Dataset Columns: Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')
Credits Dataset Columns: Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   i

In [22]:
import pandas as pd

# Load datasets
movies_df = pd.read_csv("tmdb_5000_movies.csv")
credits_df = pd.read_csv("tmdb_5000_credits.csv")

# Rename 'movie_id' to 'id' in credits_df
credits_df.rename(columns={'movie_id': 'id'}, inplace=True)

# Merge datasets
movies_df = movies_df.merge(credits_df, on='id', how='inner')

# Rename 'title_x' or 'title_y' correctly
if 'title_x' in movies_df.columns:
    movies_df.rename(columns={'title_x': 'title'}, inplace=True)
elif 'title_y' in movies_df.columns:
    movies_df.rename(columns={'title_y': 'title'}, inplace=True)

# Function to clean text
def clean_text(text):
    return text.lower() if isinstance(text, str) else ""

# Apply text cleaning
movies_df['cleaned_overview'] = movies_df['overview'].apply(clean_text)

# Display first few rows
print(movies_df[['title', 'cleaned_overview']].head())


                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                    cleaned_overview  
0  in the 22nd century, a paraplegic marine is di...  
1  captain barbossa, long believed to be dead, ha...  
2  a cryptic message from bond’s past sends him o...  
3  following the death of district attorney harve...  
4  john carter is a war-weary, former military ca...  


In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# Load datasets
movies_df = pd.read_csv("tmdb_5000_movies.csv")
credits_df = pd.read_csv("tmdb_5000_credits.csv")

# Rename 'movie_id' to 'id' in credits_df
credits_df.rename(columns={'movie_id': 'id'}, inplace=True)

# Merge datasets
movies_df = movies_df.merge(credits_df, on='id', how='inner')

# Fix 'title' column name if needed
if 'title_x' in movies_df.columns:
    movies_df.rename(columns={'title_x': 'title'}, inplace=True)
elif 'title_y' in movies_df.columns:
    movies_df.rename(columns={'title_y': 'title'}, inplace=True)

# Function to clean text
def clean_text(text):
    return text.lower() if isinstance(text, str) else ""

# Apply text cleaning
movies_df['cleaned_overview'] = movies_df['overview'].apply(clean_text)

# TF-IDF Vectorizer to convert text into numerical form
vectorizer = TfidfVectorizer(stop_words='english')
movie_vectors = vectorizer.fit_transform(movies_df['cleaned_overview'].fillna(""))  # Handle missing values

# Compute cosine similarity
cosine_sim = cosine_similarity(movie_vectors, movie_vectors)

# Function to recommend similar movies
def recommend_movies(movie_title, top_n=5):
    if 'title' not in movies_df.columns:
        return "Error: 'title' column not found in dataset."

    matches = difflib.get_close_matches(movie_title, movies_df['title'].dropna().tolist(), n=1)
    if not matches:
        return "Sorry, I couldn't find that movie. Please try another."
    
    movie_index = movies_df[movies_df['title'] == matches[0]].index[0]
    similarity_scores = list(enumerate(cosine_sim[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    recommended_movies = [movies_df.iloc[i[0]]['title'] for i in similarity_scores]
    return recommended_movies

# Test the recommendation function
print(recommend_movies("Inception"))


['Cypher', 'Mission: Impossible - Rogue Nation', 'Stone Cold', 'Central Intelligence', 'Pitch Perfect 2']


In [26]:
import pandas as pd
import ast  # To safely convert string to list

# Load datasets
movies_df = pd.read_csv("tmdb_5000_movies.csv")
credits_df = pd.read_csv("tmdb_5000_credits.csv")

# Rename 'movie_id' to 'id' in credits_df
credits_df.rename(columns={'movie_id': 'id'}, inplace=True)

# Merge datasets
movies_df = movies_df.merge(credits_df, on='id', how='inner')

# Fix 'title' column if needed
if 'title_x' in movies_df.columns:
    movies_df.rename(columns={'title_x': 'title'}, inplace=True)
elif 'title_y' in movies_df.columns:
    movies_df.rename(columns={'title_y': 'title'}, inplace=True)

movies_df = movies_df.dropna(subset=['title'])

movies_df['crew'] = movies_df['crew'].fillna('[]')  
def get_director(movie_name):
    if 'title' not in movies_df.columns:
        return "Error: 'title' column not found in dataset."

    match = movies_df[movies_df['title'].str.lower() == movie_name.lower()]
    if match.empty:
        return "Sorry, movie not found."

    try:
        crew_list = ast.literal_eval(match.iloc[0]['crew'])  
    except (ValueError, SyntaxError):
        return "Error processing crew data."

    director = [crew['name'] for crew in crew_list if crew.get('job') == 'Director']
    return director[0] if director else "Director information not available."

# Test the function
print(get_director("Inception"))


Christopher Nolan


In [30]:
def movie_chatbot():
    print("Hello! I'm your movie chatbot. Ask me for recommendations or movie details!")
    while True:
        user_input = input("You: ").strip().lower()  
        
        if user_input in ['exit', 'bye', 'quit']:
            print("Chatbot: Goodbye!")
            break
        
        elif "recommend" in user_input:
            movie_name = input("Enter a movie you like: ").strip()
            recommendations = recommend_movies(movie_name)
            print("Chatbot: You might like these movies:", recommendations)
        
        elif "who directed" in user_input:
            words = user_input.split()
            movie_name = " ".join(words[words.index("directed") + 1:]) 
            if movie_name:
                director = get_director(movie_name)
                print("Chatbot:", director)
            else:
                print("Chatbot: Please specify a movie. Example: 'Who directed Inception?'")
        
        else:
            print("Chatbot: I can recommend movies or provide details. Try asking for a recommendation!")

# Run chatbot
movie_chatbot()


Hello! I'm your movie chatbot. Ask me for recommendations or movie details!


You:  Can you recommend a movie?
Enter a movie you like:  Inception


Chatbot: You might like these movies: ['Cypher', 'Mission: Impossible - Rogue Nation', 'Stone Cold', 'Central Intelligence', 'Pitch Perfect 2']


You:  Can you recommend a movie?
Enter a movie you like:  Annabelle


Chatbot: You might like these movies: ['Fish Tank', 'Loving Annabelle', 'If I Stay', 'The Princess Diaries 2: Royal Engagement', 'The Dress']


You:  exit


Chatbot: Goodbye!
